# CIAN.ru

In [17]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [18]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

# Цена

In [19]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

# Координаты

In [20]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

# Число комнат

In [21]:
def getRooms(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    return room_number

## Этаж/всего этажей

In [22]:
def getFloor_Nfloors(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    Nfloors=-1
    k=0
    for i in re.split('\n|\xa0|/| ', re.split('Этаж:|Тип дома', table)[1]):
        if i.isdigit(): 
            if k==0: 
                floor=i
                k=k+1
            else:
                Nfloors=i
                k=k+1
        if k==2 : break
    if Nfloors==-1: return int(floor), np.NaN #если не указано кол-во этажей всего, то тогда вернём пустое значение
    else: return int(floor), int(Nfloors)

# Totsp (общая площадь квартиры)


In [23]:
def getTotsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    r=re.split('Общая площадь:|Площадь комнат', table)[1]
    r=r.replace(',','.')
    totsp=re.split('\n|\xa0| ',r)[-6]
    return float(totsp)

# Жилая площадь

In [24]:
def getLivesp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    r=re.split('Жилая площадь:|Площадь кухни', table)[1]
    r=r.replace(',','.')
    livesp=re.split('\n|\xa0| ',r)[-6]
    if '–' in re.split('\n|\xa0| ',r)  : livesp=np.NaN
    return livesp

# Площадь кухни

In [25]:
def getKitsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    r=re.split('санузлов:|Площадь кухни|Cанузел', table)[1]
    r=r.replace(',','.')
    kitsp=re.split('\n|\xa0| ',r)[2]
    if '–' in re.split('\n|\xa0| ',r)  : kitsp=np.NaN
    return kitsp

# Расстояние до метро

In [26]:
def getMetrdist(flat_page):
    place=flat_page.find('div',attrs={'class':'object_descr_metro'})
    place=html_stripper(place)
    time=-1
    for i in re.split('\n|\xa0|/| ', place):
            if (i.isdigit()) and(i!='1905'): 
                time=i
                break
    if time==-1: return np.NaN
    else: return time


# # Walk(время до метро считается пешком или на транспорте)

In [27]:
def getWalk(flat_page):
    place=flat_page.find('div',attrs={'class':'object_descr_metro'})
    place=html_stripper(place)
    walk=-1
    for i in re.split('\n|\xa0|/| ', place):
        if 'пешком' in i : 
            walk=1
            break
        if 'транспорт' in i:
            walk=0
            break
        if 'машин' in i:
            walk=0
            break
    if walk==-1 : return np.NaN # если не сказано ничего насчёт пешком/на машине, то вернём пустое значение
    else: return walk

# Телефон

In [28]:
def getTel(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    #print(table)
    if 'Телефон' in table: 
        for i in re.split('\n|\xa0|/| ', re.split('Телефон:|Вид из окна', table)[1]):
            if 'да' in i:
                tel=1
                break
            else: tel=0
    else: tel=0 #если про телефон ничего не указано, значит его нет
    return tel

## Новостройка или вторичка

In [29]:
def getNew(flat_page):
    table=flat_page.find('table', attrs = {'class':'object_descr_props'})
    table=html_stripper(table)
    new=-1
    if 'вторич' in table: new=0
    if 'новостро' in table: new=1
    if new==-1: return np.NaN #если про этот признак ничего не указано, то вернём пустое значение
    else: return new


# Тип дома(монолит, кирпичный, жб и другие)

In [30]:
def getBrick(flat_page):
    table=flat_page.find('table', attrs = {'class':'object_descr_props'})
    table=html_stripper(table)
    brick=0
    if 'кирпич' in table: brick=1
    if 'монолит' in table: brick=1
    if  'жб' in table: brick=1
    return brick

# Балкон/лоджия

In [31]:
def getBal(flat_page):
    table=flat_page.find('table', attrs = {'class':'object_descr_props'})
    table=html_stripper(table)
    bal=0
    if 'балк.' in table: bal=1
    if 'лодж.' in table: bal=1
    return bal

# Расстояние до центра

In [32]:
def getDist(flat_page):
    exactlylat=55.755826
    exactlylon=37.6173
    dist=np.sqrt((111.3*np.cos(getCoords(flat_page)[0])*(getCoords(flat_page)[1]-exactlylon))**2+(111.0*np.cos(exactlylon)*(getCoords(flat_page)[0]-exactlylat))**2)
    return dist

## Собираем всё вместе

In [33]:
flat=pd.DataFrame(columns=['District','N','Rooms','Price','Totsp','Livesp','Kitsp','Dist','Metrdist','Walk','Brick','Tel','Bal','Floor','Nfloors','New'])

In [34]:
def District(district,k,flat):
    links = []
    for page in range(1, 30):
        page_url =  district.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
    count = 0
    #total_flat=3
    for i in range(len(links)):
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[i]) + '/'
        #flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')    

        to_append = {'District':k,'N':k*812+i+1, 'Rooms':getRooms(flat_page), 'Price':getPrice(flat_page), 'Totsp':getTotsp(flat_page), 'Livesp':getLivesp(flat_page),
                     'Kitsp':getKitsp(flat_page), 'Dist':getDist(flat_page),'Metrdist':getMetrdist(flat_page), 'Walk':getWalk(flat_page),
                     'Brick':getBrick(flat_page), 'Tel':getTel(flat_page), 'Bal':getBal(flat_page),'Floor':getFloor_Nfloors(flat_page)[0],
                     'Nfloors':getFloor_Nfloors(flat_page)[1], 'New':getNew(flat_page)}
        flat = flat.append(to_append, ignore_index=True)
        count +=1 
        #if i%100==0: print('I`m not dead, I`m working! The page is {}'.format(i+1))
    return flat

## Для каждого района находим главную ссылку и для этого района заносим в таблицу данные про каждую квартиру этого района

In [ ]:
#ЦАО
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,0,flat)
#САО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,1,flat)
#CВАО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,2,flat)
#ВАО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,3,flat)
#ЮВАО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,4,flat)
#ЮАО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,5,flat)
#ЮЗАО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,6,flat)
#ЗАО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,7,flat)
#СЗАО
district='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
flat=District(district,8,flat)


## Записываем в csv файл

In [22]:

import csv
with open('Flats.csv', 'w',newline='') as csvfile:
    fieldnames = ['District','N','Rooms','Price','Totsp','Livesp','Kitsp','Dist','Metrdist','Walk','Brick','Tel','Bal','Floor','Nfloors',
                  'New']

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    n=flat.shape[0]
    writer.writeheader()
    for i in range(n):
        writer.writerow({'District':flat['District'][i],
                         'N':flat['N'][i],
                         'Rooms':flat['Rooms'][i],
                         'Price':flat['Price'][i],
                         'Totsp':flat['Totsp'][i],
                         'Livesp':flat['Livesp'][i],
                         'Kitsp':flat['Kitsp'][i],
                         'Dist':flat['Dist'][i],
                         'Metrdist':flat['Metrdist'][i],
                         'Walk':flat['Walk'][i],
                         'Brick':flat['Brick'][i],
                         'Tel':flat['Tel'][i],
                         'Bal':flat['Bal'][i],
                         'Floor':flat['Floor'][i],
                         'Nfloors':flat['Nfloors'][i],
                         'New':flat['New'][i],
                         })